# original data

In [ ]:
import pandas as pd
import numpy as np

step to create the data:
- make sure that in the folder `raw_data` there is a file `NYCTAXI202401-202403_OD.zip`
- cd to the folder `raw_data`
- linux shell `unzip NYCTAXI202401-202403_OD.zip`
- run the following notebook
- the output will be saved in the folder `data`, which will have two folders `total` and `timeperiod`.

In [ ]:
import os
if not os.path.exists("data"):
    os.makedirs("data")
if not os.path.exists("data/timeperiod"):
    os.makedirs("data/timeperiod")
if not os.path.exists("data/total"):
    os.makedirs("data/total")

In [ ]:
geo = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.geo')
print(geo.shape)
geo.head( )

In [ ]:
geo["type"].value_counts()

In [ ]:
rel = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.rel')
print(rel.shape)
rel.head()

In [ ]:
od = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.od')#.drop(["type"],axis=1)
print(od.shape)
od.head()

In [ ]:
# od["flow"].value_counts().to_frame().reset_index()

# Get two kind of network

## Network group by time interval

        早高峰时段（7:00 - 9:00）	上班、上学的高峰期。
        上午时段（9:00 - 12:00）	早高峰过后到中午的时间段。
        午间时段（12:00 - 14:00）	中午吃饭和午休时间。
        下午时段（14:00 - 17:00）	午休结束到晚高峰前的时间段。
        晚高峰时段（17:00 - 19:00）	下班、下学的高峰期。
        晚间时段（19:00 - 7:00）	晚高峰过后到第二天早高峰前的时间段。


In [8]:
od["time"] = pd.to_datetime(od["time"])
od["hour"] = od["time"].dt.hour
od

In [ ]:
# Define the boundaries of the time periods
bins = [0, 7, 9, 12, 14, 17, 19, 24]

# Define the labels for the time periods
labels = ['Night', 'Morning Rush', 'Morning', 'Noon', 'Afternoon', 'Evening Rush', 'Night']
# num_labels = [0,1,2,3,4,5,0]

tmp = od.copy(deep =True)
tmp['time_period'] = pd.cut(tmp['hour'], bins=bins, labels=labels, right=False,ordered=False)

In [ ]:
for i in labels:
    tmp = tmp.groupby("time_period").get_group("Morning Rush")
    tmp.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()
    tmp["cost"] = rel["cost"]
    tmp.to_csv(f"data/timeperiod/NYCTAXI_OD_{i}.csv")

In [ ]:
# od_flow_nozero = od[od["flow"]>0].reset_index(drop=True)

# # Define the boundaries of the time periods
# bins = [0, 7, 9, 12, 14, 17, 19, 24]

# # Define the labels for the time periods
# labels = ['Night', 'Morning Rush', 'Morning', 'Noon', 'Afternoon', 'Evening Rush', 'Night']
# # num_labels = [0,1,2,3,4,5,0]

# # Create a new column that groups hours into time periods
# od_flow_nozero['time_period'] = pd.cut(od_flow_nozero['hour'], bins=bins, labels=labels, right=False,ordered=False)

# # Use the groupby function to group by the new column
# od_flow_nozero_agg = od_flow_nozero.groupby(["time_period","origin_id","destination_id"]).agg({"flow": ["count","sum","mean"]}).reset_index()

# od_flow_nozero_agg = od_flow_nozero_agg.loc[od_flow_nozero_agg["flow"]["count"]!=0,:]

# od_flow_nozero_f = od_flow_nozero_agg.copy(deep=True)
# od_flow_nozero_f.columns = ["time_period","origin_id","destination_id","count","sum","mean"]
# od_flow_nozero_f.drop(["count","sum"],axis=1,inplace=True)
# od_flow_nozero_f.rename(columns={"mean":"mean_flow"},inplace = True)
# od_flow_nozero_f

# od_flow_nozero_f.to_csv("data/NYCTAXI_OD_timeperiod.csv",index=False)


## Total flow network

agg od by time   -> data/NYCTAXI_OD_agg.csv

In [ ]:
od_agg = od.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()

In [ ]:
od_agg

In [ ]:
# od_agg.to_csv("data/NYCTAXI_OD_agg.csv",index=False)

join od_agg and rel

In [ ]:
# od_agg = pd.read_csv("data/NYCTAXI_OD_agg.csv")
od_f = od_agg.copy(deep = True)

od_f["cost"] = rel["cost"]

In [ ]:
od_f.to_csv("data/total/NYCTAXI_OD_Total.csv",index=False)

In [ ]:
od_f.head()

## add location to the od_f --- not used

In [ ]:
# od_f_merge = pd.merge(od_f,geo,left_on="origin_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
#     .rename(columns={"coordinates":"origin_coordinates"})
# od_f_merge

In [ ]:
# od_f_merge = pd.merge(od_f_merge,geo,left_on="destination_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
#     .rename(columns={"coordinates":"destination_coordinates"})
# od_f_merge